<a href="https://colab.research.google.com/github/DFwJZ/Real-time_Bot_Detection/blob/main/fake_tweets2015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from collections import Counter

tqdm.pandas()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
root_path = '/content/drive/MyDrive/Bot_detection/cresci tweets/cresci 2015'

In [ ]:
#fake tweets
FSF = pd.read_csv(f"{root_path}/FSF_tweets.csv", encoding = 'latin_1')
INT = pd.read_csv(f"{root_path}/INT_tweets.csv", encoding = 'latin_1')
TWT = pd.read_csv(f"{root_path}/TWT_tweets.csv", encoding = 'latin_1')

<ipython-input-4-8601b5d5b802>:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  INT = pd.read_csv(f"{root_path}/INT_tweets.csv", encoding = 'latin_1')


In [ ]:
# human tweets
TFP = pd.read_csv(f"{root_path}/TFP_tweets.csv", encoding = 'latin_1')
E13 = pd.read_csv(f"{root_path}/E13_tweets.csv", encoding = 'latin_1')

In [ ]:
FSF.shape, INT.shape, TWT.shape, TFP.shape, E13.shape

((22910, 19), (58925, 19), (114192, 19), (563693, 19), (2068037, 19))

In [ ]:
df_fake = pd.concat([FSF, INT, TWT])
df_genuine = pd.concat([TFP, E13])

df_fake.shape, df_genuine.shape

((196027, 19), (2631730, 19))

In [ ]:
unique_fake_accounts = len(df_fake['user_id'].unique())
unique_genuine_accounts = len(df_genuine['user_id'].unique())
print(f'unique fake accounts: {unique_fake_accounts}\nunique genuine accounts: {unique_genuine_accounts}')

unique fake accounts: 3202
unique genuine accounts: 1946


In [ ]:
df_fake.tail()

In [ ]:
df_genuine.head(20)

### Aggregates all tweets to the account they belong to

In [ ]:
df_genuine['text'] = df_genuine['text'].astype(str)
df_genuine_aggregated_tweets = df_genuine.groupby('user_id')['text'].progress_apply(' '.join).reset_index()

100%|██████████| 1946/1946 [00:00<00:00, 3049.34it/s]


In [ ]:
df_fake['text'] = df_fake['text'].astype(str)
df_fake_aggregated_tweets = df_fake.groupby('user_id')['text'].progress_apply(' '.join).reset_index()

100%|██████████| 3202/3202 [00:00<00:00, 38756.91it/s]


In [ ]:
df_fake_aggregated_tweets.shape, df_genuine_aggregated_tweets.shape

((3202, 18), (1946, 18))

## Feature Engineering

In [ ]:
# df_fake_aggregated_tweets['tweet_length'] = df_fake_aggregated_tweets['text'].apply(len)
# df_genuine_aggregated_tweets['tweet_length'] = df_genuine_aggregated_tweets['text'].apply(len)

# avg_length_df_fake_aggregated_tweets = df_fake_aggregated_tweets['tweet_length'].mean()
# avg_length_df_genuine_aggregated_tweets = df_genuine_aggregated_tweets['tweet_length'].mean()

# avg_length_df_fake_aggregated_tweets, avg_length_df_genuine_aggregated_tweets

In [ ]:
# df_fake_aggregated_tweets.head()

In [ ]:
def get_word_frequencies(tweet_text):
    words = re.findall(r'\b\w+\b', tweet_text.lower())
    word_frequency = dict(Counter(words))

    # Filter out words with frequency less than 2
    filtered_frequency = {word: freq for word, freq in word_frequency.items() if freq > 2}

    # Sort by frequency in descending order
    sorted_frequency = dict(sorted(filtered_frequency.items(), key=lambda item: item[1], reverse=True))
    return sorted_frequency

In [ ]:
# df_fake_aggregated_tweets['word_frequencies'] = df_fake_aggregated_tweets['text'].progress_apply(get_word_frequencies)
# df_genuine_aggregated_tweets['word_frequencies'] = df_genuine_aggregated_tweets['text'].progress_apply(get_word_frequencies)

100%|██████████| 1946/1946 [00:24<00:00, 79.47it/s] 


In [ ]:
# df_fake_aggregated_tweets.head()

In [ ]:
def convert_timestamp_to_features(timestamp_str):
    try:
        if timestamp_str[-1] == 'L' and timestamp_str[:-1].isdigit():
            timestamp_str = timestamp_str[:-1]  # Remove the 'L' suffix
            timestamp_str = int(timestamp_str) / 1000
            dt = pd.to_datetime(timestamp_str, unit='s')
        else:
            dt = pd.to_datetime(timestamp_str, format="%a %b %d %H:%M:%S %z %Y")

        return dt

    except ValueError:
        print(f"ValueError for timestamp: {timestamp_str}")
        return None, None, None

In [ ]:
# # Apply the conversion function to each timestamp
# df_fake['datetime'] = df_fake['created_at'].progress_apply(convert_timestamp_to_features)

# df_fake['hour_of_day'] = df_fake['datetime'].dt.hour
# df_fake['day_of_week'] = df_fake['datetime'].dt.dayofweek
# df_fake['month'] = df_fake['datetime'].dt.month

In [ ]:
# df_fake_aggregated_tweets.head()

In [ ]:
# # Time between tweets(how frequenty)

# # Calculate time between consecutive tweets for each user
# df_fake['time_between_tweets'] = df_fake.groupby('user_id')['datetime'].diff().dt.total_seconds()

# # Calculate average time between tweets for each user
# average_time_between_tweets = df_fake.groupby('user_id')['time_between_tweets'].mean()
# df_fake_aggregated_tweets = df_fake_aggregated_tweets.merge(average_time_between_tweets, how='left', left_on='user_id', right_index=True)


In [ ]:
# # Mmost active period of the day for each user

# df_fake['period_of_day'] = pd.cut(df_fake['hour_of_day'], bins=[0, 6, 12, 18, 24], labels=['Night', 'Morning', 'Afternoon', 'Evening'], right=False)
# most_active_period_of_day = df_fake.groupby('user_id')['period_of_day'].agg(lambda x: x.value_counts().index[0])
# df_fake_aggregated_tweets = df_fake_aggregated_tweets.merge(most_active_period_of_day, how='left', left_on='user_id', right_index=True)

In [ ]:
# # Proportion of tweets made on weekdays or weekends for each user

# df_fake['is_weekday'] = df_fake['day_of_week'] < 5
# weekday_tweet_proportion = df_fake.groupby('user_id')['is_weekday'].mean()
# df_fake_aggregated_tweets = df_fake_aggregated_tweets.merge(weekday_tweet_proportion, how='left', left_on='user_id', right_index=True)


In [ ]:
# # Group by user_id and calculate features
# # !!!!average hour of the day tells us the a continuous numerical information, eg. user tweets between 3-9 pm would have a average value of 6pm!!!

# df_fake_grouped = df_fake.groupby('user_id').agg({'hour_of_day': 'mean',
#                                                   'day_of_week': lambda x:x.value_counts().index[0],
#                                                   'month': lambda x:x.value_counts().index[0],
#                                                   'retweet_count': ['mean', 'sum'],
#                                                   'reply_count': ['mean', 'sum'],
#                                                   'favorite_count': ['mean', 'sum'],
#                                                   'num_hashtags': ['mean', 'sum'],
#                                                   'num_urls': ['mean', 'sum'],
#                                                   'num_mentions': ['mean', 'sum']})

# # Flatten the multi-index columns
# df_fake_grouped.columns = ['_'.join(col).rstrip('_') for col in df_fake_grouped.columns.values]

# # Merge the timestamp-based features into df_fake_aggregated_tweets
# df_fake_aggregated_tweets = df_fake_aggregated_tweets.merge(df_fake_grouped, how='left', left_on='user_id', right_index=True)


In [ ]:
# df_fake_aggregated_tweets.head()

In [ ]:
# Generic feature extraction
def tweet_data_feature_extraction(df):
    # Word frequencies
    df['word_frequencies'] = df['text'].progress_apply(get_word_frequencies)

    # Tweet length
    df['tweet_length'] = df['text'].apply(len)

    # Usage Acitivity
    df['datetime'] = df['created_at'].progress_apply(convert_timestamp_to_features)
    df['hour_of_day'] = df['datetime'].dt.hour
    df['day_of_week'] = df['datetime'].dt.dayofweek
    df['month'] = df['datetime'].dt.month

    df['time_between_tweets'] = df.groupby('user_id')['datetime'].diff().dt.total_seconds()

    df['period_of_day'] = pd.cut(df['hour_of_day'], # Night: 0, Morning: 1, Afternoon: 2, Evening: 3
                                 bins=[0, 6, 12, 18, 24],
                                 labels=['0', '1', '2', '3'],
                                 right=False)

    df['is_weekday'] = df['day_of_week'] < 5

    df_grouped = df.groupby('user_id').agg({
        'text': ' '.join,
        'hour_of_day': 'mean',
        'day_of_week': lambda x:x.value_counts().index[0],
        'month': lambda x:x.value_counts().index[0],
        'retweet_count': ['mean', 'sum'],
        'reply_count': ['mean', 'sum'],
        'favorite_count': ['mean', 'sum'],
        'num_hashtags': ['mean', 'sum'],
        'num_urls': ['mean', 'sum'],
        'num_mentions': ['mean', 'sum'],
        'time_between_tweets': 'mean',
        'period_of_day': lambda x: x.value_counts().index[0],
        'is_weekday': 'mean'
    })

    # Flatten the multi-index columns
    df_grouped.columns = ['_'.join(col).rstrip('_') for col in df_grouped.columns.values]

    return df_grouped

In [ ]:
df_genuine_aggregated_tweets_total = tweet_data_feature_extraction(df_genuine)
df_fake_aggregated_tweets_total = tweet_data_feature_extraction(df_fake)


100%|██████████| 196027/196027 [01:10<00:00, 2761.08it/s]


In [ ]:
df_genuine_aggregated_tweets_total.columns = df_genuine_aggregated_tweets_total.columns.str.replace('_<lambda>', '')
df_fake_aggregated_tweets_total.columns = df_fake_aggregated_tweets_total.columns.str.replace('_<lambda>', '')


In [ ]:
df_fake_aggregated_tweets_total.to_pickle(f"{root_path}/fake_tweets_df.pkl")
df_genuine_aggregated_tweets_total.to_pickle(f"{root_path}/genuine_tweets_df.pkl")

In [ ]:
df_fake_aggregated_tweets_total = pd.read_pickle(f"{root_path}/fake_tweets_df.pkl")
df_genuine_aggregated_tweets_total = pd.read_pickle(f"{root_path}/genuine_tweets_df.pkl")


In [ ]:
df_fake_aggregated_tweets_total.shape, df_genuine_aggregated_tweets_total.shape

((3202, 20), (1946, 20))

In [ ]:
df_fake_aggregated_tweets_total.head()

,user_id,hour_of_day_mean,day_of_week,month,retweet_count_mean,retweet_count_sum,reply_count_mean,reply_count_sum,favorite_count_mean,favorite_count_sum,num_hashtags_mean,num_hashtags_sum,num_urls_mean,num_urls_sum,num_mentions_mean,num_mentions_sum,time_between_tweets_mean,period_of_day,is_weekday_mean,text
0,10935572,16.190647,4,8,14.879496,8273,1.910072,1062,0.942446,524,0.048561,27,0.183453,102,0.352518,196,9.777748e+02,3,0.679856,https://t.co/iocNIgHxXH. @LovesOfaLDNgirl her...
1,16119337,7.104225,0,12,1844.036620,654633,0.016901,6,0.019718,7,1.000000,355,0.033803,12,0.140845,50,-4.881397e+04,0,0.983099,RT @ARTEM_KLYUSHIN: - Ð¯ ÑÑÑ Ð² ÐÐµÑÐ»Ð¸Ð...
2,16753788,14.566667,2,4,0.000000,0,0.033333,1,0.000000,0,0.233333,7,0.266667,8,0.300000,9,-4.717938e+06,3,0.833333,"After failed trials, J&amp;J pivots to early-s..."
3,17640121,12.250000,2,10,52.600000,1052,0.800000,16,5.100000,102,0.250000,5,0.050000,1,0.650000,13,-6.130340e+06,0,0.750000,RT @Paravosperra_: #NoGarpa que se viva enojan...
4,17656600,12.228758,3,1,55.398693,16952,2.395425,733,4.794118,1467,0.352941,108,0.349673,107,1.225490,375,-4.257927e+05,3,0.800654,I just donated to @NextStepLa for his bday and...
